In [1]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()  # .env 파일의 환경변수 로드

api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)

def generate_user_questions_prompt(task_name: str, task_summary: str, context: str) -> list:
    return [
        {
            "role": "system",
            "content": (
                "당신은 실제로 어떤 민원을 넣으려는 일반 시민입니다. 아래의 사무명, 사무개요, 그리고 민원안내 정보(context)를 참고하여, "
                "현실적인 배경이 있는 질문을 5개 작성하세요.\n\n"
                "각 질문은 반드시 다음 조건을 따라야 합니다:\n"
                "1. 실제 민원인의 처지에서 상황을 간단히 설명할 것\n"
                "2. 사무명에 포함된 핵심 키워드(예: 승강기, 농지, 건축물 등)를 반드시 문장 안에 포함할 것\n"
                "3. 질문은 아래 context에 포함된 항목의 정보만을 바탕으로 작성할 것 (context에 없는 정보는 언급하지 말 것)\n"
                "4. 질문만 읽어도 어떤 사무에 대한 내용인지 명확히 추정할 수 있을 것\n"
                "5. 막연한 표현(예: '서류', '신청서', '절차')만으로 구성된 질문은 지양할 것\n\n"
                "예시:\n"
                "- 사용 중이던 승강기에 대해 부과된 과징금을 분할 납부하려면 어떤 서류가 필요한가요?\n"
                "- 유기동물 인도 요청을 하려고 하는데, 농림축산검역본부 동물보호과에 어떻게 신청하나요?\n"
                "- 국민안전처 승강기정책과로 승강기 과징금 관련 신청서를 제출하려면 어떤 양식이 필요한가요?\n"
                "- 어업허가권 명의변경 신청을 하려면 수수료가 얼마나 드는지 궁금합니다.\n\n"
                "위 예시처럼, 질문은 반드시 구체적이며 context 안의 정보 기반이어야 합니다. "
                "이제 아래 정보를 참고하여 질문을 작성하세요."
            )
        },
        {
            "role": "user",
            "content": (
                f"[사무명]: {task_name}\n"
                f"[사무개요]: {task_summary}\n\n"
                f"[context 정보]:\n{context}"
            )
        }
    ]


def get_generated_questions(task_name, task_summary, context, idx=None, total=None):
    if idx is not None and total is not None:
        print(f"[{idx}/{total}] {task_name} 처리 중...")
    else:
        print(f"{task_name} 처리 중...")

    messages = generate_user_questions_prompt(task_name, task_summary, context)
    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        temperature=0.3
    )
    raw_output = response.choices[0].message.content
    return [q.strip("-• ").strip() for q in raw_output.strip().split("\n") if q.strip()]

In [ ]:
import pandas as pd

qa_pairs = []

original_df = pd.read_excel("행정안전부_민원안내정보_20250327.xlsx")

for i in range(0, 3):
    print(f"[{i}/2999] 처리 중...")
    row = original_df.iloc[i]

    #사무명과 사무개요 유효성 확인,
    if isinstance(row.get("사무명"), str) and isinstance(row.get("사무개요"), str):
        task_name = row["사무명"]
        task_summary = row["사무개요"]

        # context 정보 구성
        context = (
            f"- 사무명: {row['사무명']}\n"
            f"- 사무개요: {row['사무개요']}\n"
            f"- 신청유형: {row.get('신청유형', '')}\n"
            f"- 신청자 자격: {row.get('신청자 자격', '')}\n"
            f"- 수수료: {row.get('수수료', '')}\n"
            f"- 수수료내역: {row.get('수수료내역', '')}\n"
            f"- 소관부처: {row.get('소관부처', '')}\n"
            f"- 담당부서: {row.get('담당부서', '')}\n"
            f"- 접수방법: {row.get('접수방법', '')}\n"
            f"- 접수/처리: {row.get('접수/처리', '')}\n"
            f"- 근거법령: {row.get('근거법령', '')}\n"
            f"- 신청서: {row.get('신청서', '')}\n"
            f"- 구비서류: {row.get('구비서류', '')}"
        )

        # GPT 호출로 질문 생성 (예: OpenAI API 등 사용)
        questions = get_generated_questions(task_name, task_summary, context)

        # 질문 저장
        for q in questions:
            print(f"\n질문: {q}")
            qa_pairs.append({
                "사무명": task_name,
                "질문": q,
            })

#DataFrame으로 저장,
new_df = pd.DataFrame(qa_pairs)

In [3]:
import pandas as pd

original_df = pd.read_excel("행정안전부_민원안내정보_20250327.xlsx")

original_df


,분류번호,사무명,사무개요,신청유형,신청자 자격,수수료,수수료내역,소관부처,담당부서,접수방법,접수/처리,근거법령,신청서,발급물 내용,구비서류
0,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN
1,9740000-0009,공탁신청(공탁물지급청구 포함),"공탁 신청, 공탁금의 지급을 위한 민원입니다. 공탁은 공탁자가 법령에 규정된 원인에...",승인,누구나신청가능,수수료 없음,수수료 없음,대법원,사법등기국,"방문, 인터넷",1. 유형없음(총 0일)\n*접수:지방법원.지원(0일)\n*경유:\n*협의:\n*처...,"1. 공탁법(제4조, 제9조) \n2. 공탁규칙(제20조, 제21조, 제23조, 제...",금전공탁서(변제 등) 공탁사무 문서양식에 관한 예규 : 별지서식 1호의 1\n,NaN,"1. 유형없음\n< 민원인 제출서류 >\n(1) 인감증명서 신분증 가압류,압류결정문..."
2,9740000-0008,친양자입양취소신고,가정법원의 허가를 얻어 친양자 입양을 취소하기 위해 신청하는 민원입니다.,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음( 즉시)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*처리...,"1. 가족관계의 등록 등에 관한 법률(제69조, 제70조)","제9호, 친양자 입양취소신고서\n",NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 친양자입양취소당사자의 가족관계등록부...
3,9740000-0007,친양자파양신고,가정법원의 허가를 얻어 친양자입양을 파양하고자 하는 사람이 재판의 확정일부터 1개월...,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음( 지체없이)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*...,1. 가족관계의 등록 등에 관한 법률(제69조),"제7호, 친양자파양신고서\n",NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 친양자파양당사자의 가족관계등록부의 ...
4,9740000-0006,성·본계속사용신고,부가 혼인외 자를 인지할 때 인지 전의 자녀의 성과 본을 유지할 경우 그 취지를 가...,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음(총 0일)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*처...,1. 가족관계의 등록 등에 관한 법률(제55조 제1항),"제3호, 성·본계속사용신고서\n",NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 성·본계속사용자의 가족관계등록부의 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6017,0000000-0008,재직(퇴직·경력)증명,공공기관에서 근무하였던 자 또는 근무하고 있는 자가 경력사항을 증명 받을 때 혹은 ...,증명,본인또는대리인신청,조례별 수수료,조례별 수수료,공통,공통,"방문, 우편, 전화, FAX, 인터넷, 현장제출 및 기타",1. 경력증명서신청(총 0일)\n*접수:전행정기관(0일)\n*경유:\n*협의:\n*...,1. 공무원 인사기록ㆍ통계 및 인사사무 처리 규정(제32조) \n2. 공무원 인사·...,"재직증명서,경력증명서 공무원 인사·성과기록 및 전자인사관리 규칙 : 별지서식 42,...",NaN,1. 기본서류\n< 민원인 제출서류 >\n(1) 재직증명서\n(2) 경력증명서
6018,0000000-0004,확인 (공통),각급 행정기관에서 보유하고 있는 자료 또는 공부상의 기록을 확인하기 위하여 열람 또...,확인,본인또는대리인신청,수수료 없음,수수료 없음,공통,공통,"방문, 우편, 전화, FAX, 인터넷, 현장제출 및 기타",1. 유형없음(총 0일)\n*접수:전행정기관(0일)\n*경유:\n*협의:\n*처리:...,1. 개별법령(),NaN,NaN,1. 기본서류\n< 민원인 제출서류 >\n(1) 1.확인서
6019,0000000-0003,질의 (공통),법령·제도·절차 등 행정업무에 관한 질의 또는 상담형식을 통한 설명이나 해석을 요구...,질의,누구나신청가능,수수료 없음,수수료 없음,공통,공통,"방문, 우편, 전화, FAX, 인터넷",1. 단순질의(총 7일)\n*접수:전행정기관(0일)\n*경유:\n*협의:\n*처리:...,1. 민원 처리에 관한 법률(),NaN,NaN,1. 기본서류\n< 민원인 제출서류 >\n(1) 1.질의서
6020,0000000-0002,건의 (공통),정부시책이나 행정제도 및 운영의 개선에 관한 사항을 건의하기 위한 민원사무입니다. ...,건의,누구나신청가능,수수료 없음,수수료 없음,공통,공통,"방문, 우편, 전화, FAX, 인터넷",1. 유형없음(총 14일)\n*접수:전행정기관(0일)\n*경유:\n*협의:\n*처리...,1. 민원 처리에 관한 법률(),NaN,NaN,1. 기본서류\n< 민원인 제출서류 >\n(1) 1.건의서


In [8]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()  # .env 파일의 환경변수 로드

api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)

def generate_user_questions_prompt(task_name: str, task_summary: str, context: str) -> list:
    return [
        {
            "role": "system",
            "content": (
                "당신은 실제로 어떤 민원을 넣으려는 일반 시민입니다. 아래의 사무명, 사무개요, 그리고 민원안내 정보(context)를 참고하여, "
                "현실적인 배경이 있는 질문을 총 5개 작성하세요.\n\n"
                "각 질문은 반드시 다음 조건을 따라야 합니다:\n"
                "1. 실제 민원인의 처지에서 상황을 간단히 설명할 것\n"
                "2. 질문 하나에는 반드시 하나의 명확한 의도만 포함할 것 (두 가지 이상을 묻지 말 것)\n"
                "3. 질문 안에 사무의 성격이나 상황이 자연스럽게 드러나도록 할 것 (예: '공탁을 신청하려는데', '입양 취소하려고 하는데' 등)\n"
                "4. 사무명 또는 사무명에 포함된 용어는 직접적으로 사용하지 말고, 일상적인 표현으로 바꿔 쓸 것\n"
                "5. 질문은 context에 제공된 정보만을 바탕으로 할 것 (없는 정보는 언급하지 말 것)\n"
                "6. 질문 하나만 보더라도 어떤 민원 상황인지 추정 가능해야 하며, 막연한 표현은 지양할 것\n\n"
                "예시:\n"
                "- 채무 문제로 법원에 돈을 맡기려고 합니다. 이럴 때 인감증명서가 꼭 필요한가요?\n"
                "- 가압류 판결을 받은 상황인데, 이걸 증명하는 서류가 꼭 있어야 하나요?\n"
                "- 법원에 맡긴 돈을 다시 찾고 싶을 때 필요한 서류가 궁금합니다.\n"
                "- 사업상 문제로 법원에 일정 금액을 맡기려고 하는데, 사업자등록증 외에 뭘 더 내야 하나요?\n"
                "- 외국에 거주 중인데, 한국 법원에 돈을 맡기려면 어떤 절차를 따라야 하나요?\n\n"
                "이제 아래 정보를 참고하여 질문을 작성하세요."
            )
        },
        {
            "role": "user",
            "content": (
                f"[사무명]: {task_name}\n"
                f"[사무개요]: {task_summary}\n\n"
                f"[context 정보]:\n{context}"
            )
        }
    ]


def get_generated_questions(task_name, task_summary, context, idx=None, total=None):
    if idx is not None and total is not None:
        print(f"[{idx}/{total}] {task_name} 처리 중...")
    else:
        print(f"{task_name} 처리 중...")

    messages = generate_user_questions_prompt(task_name, task_summary, context)
    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        temperature=0.3
    )
    raw_output = response.choices[0].message.content
    return [q.strip("-• ").strip() for q in raw_output.strip().split("\n") if q.strip()]

In [10]:
import pandas as pd

qa_pairs = []

original_df = pd.read_excel("행정안전부_민원안내정보_20250327.xlsx")

for i in range(0, 5):
    print(f"[{i}/2999] 처리 중...")
    row = original_df.iloc[i]

    #사무명과 사무개요 유효성 확인,
    if isinstance(row.get("사무명"), str) and isinstance(row.get("사무개요"), str):
        task_name = row["사무명"]
        task_summary = row["사무개요"]

        # context 정보 구성
        context = (
            f"- 사무명: {row['사무명']}\n"
            f"- 사무개요: {row['사무개요']}\n"
            f"- 신청유형: {row.get('신청유형', '')}\n"
            f"- 신청자 자격: {row.get('신청자 자격', '')}\n"
            f"- 수수료: {row.get('수수료', '')}\n"
            f"- 수수료내역: {row.get('수수료내역', '')}\n"
            f"- 소관부처: {row.get('소관부처', '')}\n"
            f"- 담당부서: {row.get('담당부서', '')}\n"
            f"- 접수방법: {row.get('접수방법', '')}\n"
            f"- 접수/처리: {row.get('접수/처리', '')}\n"
            f"- 근거법령: {row.get('근거법령', '')}\n"
            f"- 신청서: {row.get('신청서', '')}\n"
            f"- 구비서류: {row.get('구비서류', '')}"
        )

        # GPT 호출로 질문 생성 (예: OpenAI API 등 사용)
        questions = get_generated_questions(task_name, task_summary, context)

        # 질문 저장
        for q in questions:
            print(f"\n질문: {q}")
            qa_pairs.append({
                "사무명": task_name,
                "질문": q,
            })

#DataFrame으로 저장,
new_df = pd.DataFrame(qa_pairs)

[0/2999] 처리 중...
토지임야대장등본교부 다량신청 처리 중...

질문: 1. 여러 개의 토지에 대한 정보를 한 번에 요청하고 싶은데, 인터넷으로 어떻게 신청해야 하는지 구체적으로 알고 싶습니다.

질문: 2. 여러 필지의 토지임야대장등본을 동시에 신청하려고 하는데, 특별히 준비해야 할 서류가 있는지 궁금합니다.

질문: 3. 여러 번지를 한꺼번에 신청할 수 있다고 들었는데, 이 경우 수수료는 따로 발생하지 않는 건가요?

질문: 4. 토지에 대한 정보를 요청할 때, 신청자 자격이 따로 없다고 하던데, 누구나 신청할 수 있는지 확인하고 싶습니다.

질문: 5. 인터넷으로 신청할 때, 어떤 절차를 따라야 하는지 자세히 안내받고 싶습니다.
[1/2999] 처리 중...
공탁신청(공탁물지급청구 포함) 처리 중...

질문: 1. 법원에 돈을 맡기려고 하는데, 이때 필요한 서류가 어떤 것인지 알고 싶습니다. 특히 인감증명서가 꼭 필요한가요?

질문: 2. 채무 변제를 위해 금전을 맡기고 싶은데, 어떤 절차를 따라야 하는지 궁금합니다. 신청서 양식은 어디서 받을 수 있나요?

질문: 3. 손해 담보를 위해 금전을 맡기려고 하는데, 주민등록등본 외에 추가로 제출해야 할 서류가 있는지 알고 싶습니다.

질문: 4. 법원에 맡긴 돈을 다른 사람에게 지급하려고 하는데, 이 경우 어떤 서류를 제출해야 하는지 궁금합니다.

질문: 5. 외국에 거주 중인데, 한국 법원에 돈을 맡기려면 어떤 절차를 따라야 하는지 알고 싶습니다. 특히 외국인등록사실증명은 어떻게 준비해야 할까요?
[2/2999] 처리 중...
친양자입양취소신고 처리 중...

질문: 1. 친양자 입양을 취소하려고 하는데, 이때 필요한 서류가 어떤 것인지 알고 싶습니다.

질문: 2. 법원에서 친양자 입양 취소를 위한 재판을 진행했는데, 이 판결문을 제출해야 하는지 궁금합니다.

질문: 3. 친양자 입양을 취소하기 위해 방문 신고를 하려고 하는데, 신분증 외에 다른 서류는 필요 없는지 확인하고 싶습니다.

질문